In [57]:
from pathlib import Path
import os 
import glob
from datetime import datetime
import re
import pandas as pd
import hashlib

In [4]:
os.getcwd()

'd:\\develop\\upnote_md_mgr\\notebook'

In [5]:
upnote_path = Path("../UpNote/General Space")

In [6]:
upnote_path

WindowsPath('../UpNote/General Space')

In [46]:
files = [os.path.join(upnote_path,f) for f in glob.glob("*.md", root_dir=upnote_path)]

In [121]:
def convtxt(path):
    s = ""
    update_dt = ""
    create_dt = ""
    cat_mode = False
    isheader = True
    categories = []
    bdytxt = []
    tags = []

    with open(path,"r", encoding='utf-8') as f:
        txt = f.readlines()

    for i, line in enumerate(txt):
        if i > 0:
            # skip first line
            s = line.rstrip()
            if s.startswith("date:"):
                s2 = s.replace("date: " ,"")
                update_dt = datetime.strptime(s2, "%Y-%m-%d %H:%M:%S")
            elif s.startswith("created: "):
                s2 = s.replace("created: ","")
                create_dt = datetime.strptime(s2, "%Y-%m-%d %H:%M:%S")
            elif s.startswith("categories:"):
                cat_mode = True
            elif s == "---":
                isheader = False
                cat_mode = False
                continue
            else:
                if cat_mode:
                    s2 = re.sub(r"^- ","",s)
                    categories.append(s2)

            # start of body parser
            if not isheader:
                if bool(re.match(r"#+ ",s)):
                    s2 = re.sub(r"#+ ","",s)
                    bdytxt.append(s2)
                elif bool(re.match(r"^[\s\*]+$",s)):
                    s2 = re.sub(r"^[\s\*]+$","",s)
                    bdytxt.append(s2)
                elif bool(re.findall(r"#\w+", s)):
                    for m in re.findall(r"#\w+", s):
                        s2 = re.sub(r"^#","",m)
                        if not s2.isdigit():
                            tags.append(s2)
                else:
                    bdytxt.append(s)
            # body
        # i > 0
    # i
    
    cat_str = "|".join(categories)
    contents = "\n".join(bdytxt)
    tags_str = "|".join(tags)
    
    fpath = os.path.basename(path)
    
    sha1 = hashlib.sha1()
    sha1.update(fpath.encode('utf-8'))
    id = sha1.hexdigest()

    yield [id, fpath, update_dt, create_dt, cat_str, tags_str, contents]


In [117]:
s = "#CAD #R \n #TES # teste # any ## test #123"
m = re.findall(r"#\w+", s)

In [120]:
x1 = [re.sub(r"^#","",x) for x in m]
[x for x in x1 if not x.isdigit()]

['CAD', 'R', 'TES']

In [119]:
x1

['CAD', 'R', 'TES', '123']

In [122]:
fp1 = upnote_path / "図番台帳.md"

In [123]:
list(convtxt(fp1))

[['b8cfc99654bee212665412c5f78ed279816e9e1b',
  '図番台帳.md',
  datetime.datetime(2023, 4, 14, 1, 4, 40),
  datetime.datetime(2023, 4, 13, 20, 4, 12),
  '仕事_Evernote',
  'CAD|R',
  '\n図番台帳\n\n深夜枠\n\n\n\n豊岡・埼玉・YFTの3種類の図番台帳を予め1つの台帳にまとめるスクリプトを書いて、\n\n変換済の台帳をまとめて検索できる仕組みを作り始めることにした。\n\n\n\n輸出管理データ検索同様RのArrowとDTパッケージを使用する。\n\n\n\ncad\\_basicsにてdrawinglistフォルダ。\n\n\n\nまずはそれぞれの台帳を無条件にコピー。\n\n生技・埼玉・YFTの台帳を統合するスクリプトを書いて、その後DTによるWEBでデータベース検索するappを作成。\n\nappについては以前作成した輸出管理検索用がほとんどそのまま使えた。\n\n\n\n2時間程度で全部完成。\n\n\n\nちなみに入力日が2050年よりも大きい物が4点あり、それぞれエクセルのデータからして間違って入力されている。\n\n\n\n改善提案に出来るはず。\n\n\n']]

In [124]:
files1 = files[:10]

In [125]:
files1

['..\\UpNote\\General Space\\- 名刺.md',
 '..\\UpNote\\General Space\\0030.md',
 '..\\UpNote\\General Space\\0042.md',
 '..\\UpNote\\General Space\\0音孔.md',
 '..\\UpNote\\General Space\\1,2番管.md',
 '..\\UpNote\\General Space\\1,3,4-2 抜差組立 中管ピン径変更.md',
 '..\\UpNote\\General Space\\1-3レバー 2.md',
 '..\\UpNote\\General Space\\1-3レバー.md',
 '..\\UpNote\\General Space\\1.5 スタイラス.md',
 '..\\UpNote\\General Space\\1.5スタイラス.md']

In [126]:
alldata = []
for f in files1:
    for output in convtxt(f):
        alldata.append(output)

In [127]:
dfm = pd.DataFrame(alldata, columns=["id", "fpath","update","created","category","tags","contents"])


In [128]:
dfm

,id,fpath,update,created,category,tags,contents
0,f370a90e365a143eaa1a83a5c518bd0c64aebcca,- 名刺.md,2017-01-21 20:01:34,2017-01-21 20:01:05,名刺,,\n\- 名刺\n\n池田 信次郎日新\n\n\n\nDSI成形機の輸送\n\n![](Fi...
1,395eede9ca9785cde411b704bba4b45fcdbfcbd6,0030.md,2014-06-05 11:06:15,2014-06-05 11:06:02,仕事_Evernote,HR869,\n0030\n\n1番管正寸切治具のベースへの取り付け孔をφ11にするようセガワにメール連...
2,c3a4cc6ab6ac8c013e2e6bd8b153ae7b1dc8750a,0042.md,2014-06-11 16:06:42,2014-06-05 11:06:15,仕事_Evernote,HR869,\n0042\n\n枝管3Aの寸切治具、浅羽ではなくYMC用だった。\n\nで、ボルト孔が小...
3,474aca1636653a21e3e48122f5015d59636c2495,0音孔.md,2017-02-09 18:02:36,2017-02-09 15:02:23,仕事_Evernote,分岐管,\n0音孔\n\nボディ測定結果を確認。\n\n大野さんにメール。\n\n\n\n結果、+0...
4,638af3f32369bad23bd055f03255660a0d14e72d,"1,2番管.md",2022-07-25 23:07:08,2022-07-25 16:07:58,仕事_Evernote,CBBB623,"\n1,2番管\n\nEMCによるナミ修正品の確認を徳富さんと見た。\n\nまだダクってるの..."
5,dc9a6495db2e985051086400018783a2aa9fd3de,"1,3,4-2 抜差組立 中管ピン径変更.md",2022-02-20 02:02:42,2022-02-20 02:02:34,仕事_Evernote,CBBB623,"\n1,3,4-2 抜差組立 中管ピン径変更\n\n中管の嵌合径が大きなクリアランスを取って..."
6,02840cf04ea03a20071f3f52cbd5f7ceab7cdd27,1-3レバー 2.md,2016-05-25 14:05:11,2016-05-25 14:05:33,仕事_Evernote,HR869,\n1-3レバー\n\n原さんから、869 1-3レバー変更の依頼がPJから来たら対応してほ...
7,d00d55b5004d2cdae7ad573ec0899d7b90757a58,1-3レバー.md,2016-07-08 14:07:00,2016-07-08 13:07:01,仕事_Evernote,HR869,\n1-3レバー\n\n室園氏。\n\n\n\n設変対応。\n\n何をやればいいのかを室園氏...
8,273d0a0b17163e868d17a9d5393680c490774bdb,1.5 スタイラス.md,2016-02-09 14:02:11,2016-02-09 14:02:10,仕事_Evernote,3次元,\n1.5 スタイラス\n\n渡邊氏から電話。新しい2本が納品されたが、棚に入れようと思った...
9,159dab4768e0d83b2b2d9876603b075296eb6cb2,1.5スタイラス.md,2016-01-21 10:01:53,2016-01-21 10:01:08,仕事_Evernote,3次元,\n1.5スタイラス\n\n昨日の破損事故を受けて、関係者にメール。\n\n今後、許可者以外...


In [81]:
dfm.to_csv("upnote_text_tmp.csv", index=False)